# VEGAS implementation by Timur Khuzin

Sources:
1. https://e-maxx.ru/bookz/files/numerical_recipes.pdf, pp. 414-416
2. http://numerical.recipes/webnotes/nr3web9.pdf

## Implementation

In [1]:
import numpy as np
import numpy.random

def vegas_integral(region, f, epochs, call_per_epoch, rand=np.random.RandomState(seed=164), steps_per_dim):
    '''
    Implements vegas integral.
    params:
        region          is matrix 2xD, where D is dimension number. region[0,:] is lower bounds, region[1,:] is upper
        f               is integrated function. Must accept numpy vector with len D and one float argument (weight)
        epochs          number of iterations of algorithm (calculations of integral)
        call_per_epoch  how many checks needed to calc integral
        rand            random float generator
        steps_per_dim   maximal number of increments per axis
        
    returns object with fields: 
        integral        float value of calculated integral
        std_deviation   standard deviation of calculated integral
        chi2a           look Chi-squared test
    '''
    dimensions = len(region[0,:])
    # make our region start from zero
    shift = region[0,:]
    region = np.copy(region)
    region[0,:] -= shift
    region[1,:] -= shift
    
    # our histograms
    X = np.zeros(dimensions, steps_per_dim)
    # initially equal in all range
    X[:,0] = 1.0
    
    mds = ndo = 1.0 # WTF?
    
    for iteration in range(epochs):
        pass
    
    
    # return result
    class answer: pass
    ans = answer()
    ans.integral = integral
    ans.std_deviation = std_deviation
    ans.chi2a = chi2a
    return ans

In [6]:
a = np.zeros((2,5))
print(a)
a[1,:] = 1
print(a)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1.]]
